In [3]:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Importing functions
import sys
sys.path.append(r'C:\\Users\\Manuel Elizaldi\\Desktop\\Learning-Testing\\PyStrava\\Scripts')
from Functions import *

In [4]:
# Specify the path to your JSON file
file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Outputs\all_workouts_json.json'

# Open the JSON file and load its contents into a Python object
with open(file_path, 'r') as json_file:
    all_workouts_json = json.load(json_file)

In [5]:
laps_df = pd.json_normalize(all_workouts_json, 'laps')

In [8]:
laps_df = laps_df[['activity.id',
                   'id',
                   'moving_time',
                   'start_date',
                   'distance',
                   'average_speed',
                   'max_speed',
                   'lap_index',
                   'split',
                   'start_index',
                   'end_index',
                   'total_elevation_gain',
                   'average_cadence',
                   'average_heartrate',
                   'max_heartrate',
                   'pace_zone']]

laps_df

,activity.id,id,moving_time,start_date,distance,average_speed,max_speed,lap_index,split,start_index,end_index,total_elevation_gain,average_cadence,average_heartrate,max_heartrate,pace_zone
0,10343078755,35601174991,428,2023-12-07T13:56:12Z,1000.00,2.34,3.378,1,1,0,81,4.4,86.5,118.2,130.0,0
1,10343078755,35601174992,452,2023-12-07T14:04:08Z,1000.00,2.21,3.199,2,2,82,191,18.0,85.6,137.2,151.0,0
2,10343078755,35601174995,414,2023-12-07T14:11:49Z,1000.00,2.42,3.978,3,3,192,275,7.6,87.0,137.8,152.0,0
3,10343078755,35601174997,7,2023-12-07T14:19:03Z,13.95,1.99,2.073,4,4,275,275,0.0,86.0,151.0,151.0,0
4,10322687345,35523250605,347,2023-12-03T13:10:45Z,1000.00,2.88,4.078,1,1,0,63,8.6,89.1,139.4,156.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,9850629317,33728782837,336,2023-09-14T21:43:29Z,1000.00,2.98,3.724,2,2,52,136,7.6,88.4,158.8,161.0,0
125,9850629317,33728782839,346,2023-09-14T21:49:02Z,1000.00,2.89,3.696,3,3,137,219,14.4,87.4,164.2,169.0,0
126,9850629317,33728782845,359,2023-09-14T21:55:02Z,1000.00,2.79,3.678,4,4,220,298,9.2,86.5,165.2,170.0,0
127,9850629317,33728782847,331,2023-09-14T22:00:56Z,1000.00,3.02,3.715,5,5,299,357,5.4,87.5,170.9,176.0,0


In [18]:
laps_df.to_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Outputs\laps_table.csv')

In [ ]:
# This function will parse the workout json, grab the relevant columns, clean the units and create a lap counter for the final dataframe
def CleanLapsJson(workout_json):
    # parsing json
    df = pd.json_normalize(workout_json)

    # Formatting distance to km 
    df['distance'] = round(df['distance']/1000,2)

    # Formatting workout time to actual minutes
    df['workout_time_min'] = round(df['moving_time']/60,2)
    df['workout_time_min'] = round(df['workout_time_min'] - (df['workout_time_min']%1) + (((df['workout_time_min']%1) * 60)/100),2)
    
    # Date in the right format
    df["start_date"] = pd.to_datetime(df['start_date']).dt.date
    # Formatting speed to km/h
    df['average_speed'] = df['average_speed'] * 3.6
    df['max_speed'] = df['max_speed'] * 3.6
    df = df.rename(columns={'id':'activity_id','average_speed':'average_speed_km/h','max_speed':'max_speed_km/h'})
    df['sport_type'] = df['sport_type'].replace({'Workout':'Functional-Cardio Workout'})
    # Creating the start and end latitude and longitude
    df[['start_lat', 'start_long']] = df['start_latlng'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))
    df[['end_lat', 'end_long']] = df['end_latlng'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))

    #Getting the columns we want
    df = df[['activity_id',
                'name',
                'start_date',
                'sport_type',
                'distance',
                'workout_time_min',
                'calories',
                'total_elevation_gain',
                'start_lat',
                'start_long',
                'end_lat',
                'end_long',
                'average_speed_km/h',
                'max_speed_km/h',
                'average_temp',
                'average_heartrate',
                'max_heartrate']]


    # Now working on laps data frame, we create 2, 1 for lap avg time and lap counter
    workout_laps = pd.json_normalize(workout_json,'laps')
    workout_laps = workout_laps[['activity.id','name','moving_time','distance','average_heartrate','max_heartrate','average_speed','max_speed']]
    workout_laps = workout_laps.rename(columns={'activity.id':'activity_id',
                                                    'name':'lap',
                                                    'moving_time':'lap_time_min',
                                                    'distance':'lap_distance',
                                                    'average_heartrate':'lap_average_heartrate',
                                                    'max_heartrate':'lap_max_heartrate_km/h',
                                                    'average_speed':'lap_average_speed_km/h',
                                                    'max_speed':'lap_max_speed'})